# Template for the data structure 

We want to all use the same data structure as below


In [1]:
import numpy as np
import pandas as pd

In [2]:
import pandas as pd

# Define columns, with key quantities stored as lists: [err-, value, err+]
columns = [
    "System Name", "RA", "Dec", "Period", "Eccentricity",
    "M1","M1_sin3i", "M2", "M2_sin3i", "q", "Mass Function",
    "Type1", "Type2", "Detection Method", "Reference", "Notes"
]


# Initialize empty DataFrame
contact_df = pd.DataFrame(columns=columns)

# Define helper function using [err-, value, err+] triplets
def add_observation(df, system_name,
                    ra, dec, period, ecc,
                    m1, m1_sin3i, m2, m2_sin3i, q, mass_func,
                    type1, type2, method, reference, notes=""):
    # use np.inf if it is a lower limit (0 if it is an upper limit)
    new_row = {
       "System Name": system_name,
        "RA": ra,                           # [err-, value, err+]
        "Dec": dec,                         # [err-, value, err+]

        "Period": period,                   # day [err-, value, err+]
        "Eccentricity": ecc,                # [err-, value, err+ ]

        "M1": m1,                           # Accretor star [err-, value, err+]
        "M1_sin3i": m1_sin3i,               # M1 sini^3 values [err-, value, err+] (= lower limit on m1)
        "M2": m2,                           # Donor (post MT 1) [err-, value, err+,]
        "M2_sin3i": m2_sin3i,               #  M2 sini^3 values [err-, value, err+] (= lower limit on m2)
        "q": q,                             # M2/M1 = donor/accretor [err-, value, err+, lower/upper limit? ]
        "Mass Function": mass_func,         # [err-, value, err+, lower/upper limit? ]

        "Type1": type1,                     # ["MS", "WD", "NS", "BH" "RG", "O", "B" ]
        "Type2": type2,                     # ["MS", "WD", "NS", "BH" "RG", "O", "B" ]
        "Detection Method": method,         # list of strings ["Xray", "RV"= Radial velocity, "EB"=Eclipsing binary, "AB" = Astrometric binary, "Other"]  
        "Reference": reference,             # ADS Bibcode    
        "Notes": notes
    }
    return pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)

# Example: Add two entries
#observations_df = add_observation(
#    observations_df,
#    "Example system",
#    [0.005, 150.025, 0.005],        # RA
#    [0.004, -45.123, 0.004],        # Dec
#    [0.2, 12.5, 0.3],               # Period in days
#    [0.02, 0.30, 0.03],             # Eccentricity
##    [0.3, 10.0, 0.4],               # M1 in Msun
#    [0.2, 2.5, 0.3],                # M2 in Msun
#    [0.03, 0.25, 0.04],             # q (mass ratio)
#    [0.01, 1e-3, 0.02],             # Mass Function
#    "MS",                            # Type1
#    "WD",                            # Type2
#    ["RV", "EB"],                    # Detection Method
##    "2025arXiv250514780L",           # Reference (use Ads Bibcode)
#   "This is an example system ",    # Notes
#)


##################
# Example edge cases

# How to add an lower limit (e.g., M1 > 5 Msun)?
#[0.0,5.0, np.inf],              
# Upper lower limit (e.g., M1 < 5 Msun)?
#[5.0, 5.0, 0.0],               
# # No error (e.g., M1 = 5 Msun)?
#[np.nan, 5.0, np.nan],            



In [3]:
contact_df = add_observation(contact_df,
    "LSS 3074", 			
    [0.0097, 201.74923546386, 0.0097],        # RA
    [0.0118, -62.03037749215, 0.0118],        # Dec
    [0.0006, 2.1852, 0.0006],               # Period in days
    [0.0, 0.0, 0.0],             # Eccentricity
    [1.4, 17.2, 1.4],               # M1 in Msun
    [np.nan,np.nan,np.nan],          # M1 sin3i
    [1.1, 14.8, 1.1],                # M2 in Msun
    [np.nan,np.nan,np.nan],             # M2sin3 
    [0.04, 0.86, 0.04],             # q (mass ratio)
    [np.nan,np.nan,np.nan],             # Mass Function
    "O6-7:(f):, Contact",           # Type1
    "O4 f +, Contact",                            # Type2
    ["RV", "EB"],                    # Detection Method
    "2017A&A...601A.133R",           # Reference (use Ads Bibcode)
    "Gaia DR3 5868409430865830912 ",    # Notes
)


In [4]:
contact_df = add_observation(contact_df,
    "MY Cam", 
    [0.0236, 059.82622297558, 0.0236],        # RA
    [0.0207, +57.23715731461, 0.0207],        # Dec
    [0.0000015, 1.1754514, 0.0000015],               # Period in days
    [0.0, 0.0, 0.0],             # Eccentricity
    [1.6, 37.7, 1.6],               # M1 in Msun
    [np.nan,np.nan,np.nan],          # M1 sin3i
    [1.4, 31.6, 1.4],                # M2 in Msun
    [np.nan,np.nan,np.nan],             # M2sin3 
    [0.04, 0.84, 0.04],             # q (mass ratio)
    [np.nan,np.nan,np.nan],             # Mass Function
    "O4.5-6V, Contact",           # Type1
    "O6V, Contact",                            # Type2
    ["RV", "EB"],                    # Detection Method
    "2014A&A...572A.110",           # Reference (use Ads Bibcode)
    "Gaia DR3 469715181320008960 ",    # Notes
)

In [5]:
contact_df = add_observation(contact_df,
    "OGLE SMC-SC10 108086", 	
    [0.0242	, 016.37750625155, 0.0242],        # RA
    [0.0219 , -72.02269945610, 0.0219],        # Dec
    [np.nan, 0.8830987, np.nan],               # Period in days
    [0.0, 0.0, 0.0],                # Eccentricity
    [1.2, 16.9, 1.2],               # M1 in Msun
    [np.nan,np.nan,np.nan],          # M1 sin3i
    [1.7, 14.3, 1.7],               # M2 in Msun
    [np.nan,np.nan,np.nan],             # M2sin3 
    [0.06, 0.85, 0.06],             # q (mass ratio)
    [np.nan,np.nan,np.nan],        # Mass Function
    "O9V, Contact",                 # Type1
    "O9.5-B0V, Contact",                            # Type2
    ["RV", "EB"],                    # Detection Method
    "2005MNRAS.357..304H",           # Reference (use Ads Bibcode)
    "Gaia DR3 4690510870334496896, SMC ",    # Notes
)

In [6]:
contact_df = add_observation(contact_df,
    "TU Mus", 
    [0.0188	, 172.79544384537, 0.0188],        # RA
    [0.0195 , -65.74225407400	, 0.0195],        # Dec
    [0.0000017, 1.3872827, 0.0000017],               # Period in days
    [0.0, 0.0, 0.0],                # Eccentricity
    [0.4, 16.7, 0.4],               # M1 in Msun
    [np.nan,np.nan,np.nan],          # M1 sin3i
    [0.4, 10.4, 0.4],               # M2 in Msun
    [np.nan,np.nan,np.nan],             # M2sin3 
    [0.009, 0.623, 0.009],             # q (mass ratio)
    [np.nan,np.nan,np.nan],        # Mass Function
    "O7V, Contact",                 # Type1
    "O8V, Contact",                            # Type2
    ["RV", "EB", "IUE"],                    # Detection Method
    "2008ApJ...681..554P",           # Reference (use Ads Bibcode)
    "Gaia DR3 5237207155697930496, HD 100213",    # Notes
)

In [7]:
contact_df = add_observation(contact_df,
    "V382 Cyg", 	
    [0.0183	, 304.69673293063, 0.0183],        # RA
    [0.0240 , +36.34054524438, 0.0240],        # Dec
    [0.000007, 1.885545, 0.000007],               # Period in days
    [0.0, 0.0, 0.0],                # Eccentricity
    [0.4, 26.1, 0.4],               # M1 in Msun
    [np.nan,np.nan,np.nan],          # M1 sin3i
    [0.3, 19.0, 0.3],               # M2 in Msun
    [np.nan,np.nan,np.nan],             # M2sin3 
    [0.005, 0.727,0.005],             # q (mass ratio)
    [np.nan,np.nan,np.nan],        # Mass Function
    "O6.5V((f)), Contact",                 # Type1
    "O6V((f)), Contact",                            # Type2
    ["RV", "EB"],                    # Detection Method
    "2017A&A...607A..82M",           # Reference (use Ads Bibcode)
    "Gaia DR3 2057530097777247744, HD 228854 ",    # Notes
)

In [8]:
contact_df = add_observation(contact_df,
    "VFTS 352", 		
    [0.0170,084.61858826377,0.0170],        # RA
    [0.0195,-69.18864491570,0.0195],        # Dec
    [0.0000004, 1.1241452, 0.0000004],               # Period in days
    [0.0, 0.0, 0.0],                # Eccentricity
    [0.3, 28.9, 0.3],               # M1 in Msun
    [np.nan,np.nan,np.nan],          # M1 sin3i
    [0.3, 28.6, 0.3],               # M2 in Msun
    [np.nan,np.nan,np.nan],             # M2sin3 
    [0.10, 0.99, 0.10],             # q (mass ratio)
    [np.nan,np.nan,np.nan],        # Mass Function
    "O4.5 V(n)((fc))z, Contact",                 # Type1
    "O5.5 V(n)((fc))z, Contact",                            # Type2
    ["RV", "EB"],                    # Detection Method
    "2015ApJ...812..102A",           # Reference (use Ads Bibcode)
    "Gaia DR3 4657678216202797440, LMC",    # Notes
)

In [9]:
contact_df = add_observation(contact_df,
    "CT Tau", 			
    [0.0210,089.70880588359,0.0210],        # RA
    [0.0182,+27.07830392924,0.0182],        # Dec
    [0.00000016, 0.66682928, 0.00000016],               # Period in days
    [0.0, 0.0, 0.0],                # Eccentricity
    [3.3, 14.25, 3.3],               # M1 in Msun
    [np.nan,np.nan,np.nan],          # M1 sin3i
    [3.4, 14.01, 3.4],               # M2 in Msun
    [np.nan,np.nan,np.nan],             # M2sin3 
    [0.003, 0.983, 0.003],             # q (mass ratio)
    [np.nan,np.nan,np.nan],        # Mass Function
    "B1V, Contact",                 # Type1
    "B1V, Contact",                            # Type2
    ["RV", "EB",],                    # Detection Method
    "2019AJ....157..111Y",           # Reference (use Ads Bibcode)
    "Gaia DR3 3430795519289951744,HD 249751",    # Notes
)

In [10]:
contact_df = add_observation(contact_df,
    "GU Mon", 			
    [0.0172,101.19525090096,0.0172],        # RA
    [0.0172,+00.22175161988,0.0172],        # Dec
    [0.00000056, 0.89664680, 0.00000056],               # Period in days
    [0.0, 0.0, 0.0],                # Eccentricity
    [0.13, 8.79, 0.13],               # M1 in Msun
    [np.nan,np.nan,np.nan],          # M1 sin3i
    [0.12, 8.58, 0.12],               # M2 in Msun
    [np.nan,np.nan,np.nan],             # M2sin3 
    [0.003, 0.976, 0.003],             # q (mass ratio)
    [np.nan,np.nan,np.nan],        # Mass Function
    "B1V, Contact",                 # Type1
    "B1V, Contact",                            # Type2
    ["RV", "EB",],                    # Detection Method
    "2019AJ....157..111Y",           # Reference (use Ads Bibcode)
    "Gaia DR3 3125506151511957120",    # Notes
)

In [11]:
contact_df = add_observation(contact_df,
    "SV Cen", 		
    [0.0235,176.98835928994	,0.0235],        # RA
    [0.0222,-60.56604234441,0.0222],        # Dec
    [0.001, 1.658, 0.001],               # Period in days
    [0.0, 0.0, 0.0],                # Eccentricity
    [0.35, 8.56, 0.35],               # M1 in Msun
    [np.nan,np.nan,np.nan],          # M1 sin3i
    [0.26, 6.05, 0.26],               # M2 in Msun
    [np.nan,np.nan,np.nan],             # M2sin3 
    [0.002, 0.71, 0.002],             # q (mass ratio)
    [np.nan,np.nan,np.nan],        # Mass Function
    "B, Contact?",                 # Type1
    "B, Contact?",                            # Type2
    ["RV", "EB",],                    # Detection Method
    "1992AJ....103..573R",           # Reference (use Ads Bibcode)
    "Gaia DR3 5335388664983921024, HD 102552,Pdot significant, thermal timescale evolution?; might not be contact but semi-detached: 2024A&A...691A.150V'",    # Notes
)

In [12]:
contact_df = add_observation(contact_df,
    "V606 Cen", 
    [0.0092,200.40113235649	,0.0092],        # RA
    [0.0110,-60.52077138747,0.0110],        # Dec
    [0.0000011, 1.4950996, 0.0000011],               # Period in days
    [0.07, 0.33, 0.07],                # Eccentricity
    [0.40, 14.70, 0.40],               # M1 in Msun
    [np.nan,np.nan,np.nan],          # M1 sin3i
    [0.22, 7.74, 0.22],               # M2 in Msun
    [np.nan,np.nan,np.nan],             # M2sin3 
    [0.0007, 0.5484, 0.0007],             # q (mass ratio)
    [np.nan,np.nan,np.nan],        # Mass Function
    "B0.5V, Contact",                 # Type1
    "B2V, Contact",                            # Type2
    ["RV", "EB",],                    # Detection Method
    "1999A&A...345..531L,2022ApJ...924...30L",           # Reference (use Ads Bibcode)
    "Gaia DR3 5869546428991037312, HD 115937; triple (P_out~88.3yr)",    # Notes
)

In [13]:
contact_df = add_observation(contact_df,
    "V701 Sco", 
    [0.0293,263.60214994049	,0.0293],        # RA
    [0.0219,-32.50444857968,0.0219],        # Dec
    [0.0000013, 0.76187385, 0.0000013],               # Period in days
    [0.0, 0.0, 0.0],                # Eccentricity
    [0.22, 9.78, 0.22],               # M1 in Msun
    [np.nan,np.nan,np.nan],          # M1 sin3i
    [0.22, 9.74, 0.22],               # M2 in Msun
    [np.nan,np.nan,np.nan],             # M2sin3 
    [0.002, 0.995, 0.002],             # q (mass ratio)
    [np.nan,np.nan,np.nan],        # Mass Function
    "B1-1.5V, Contact",                 # Type1
    "B1-1.5V, Contact",                            # Type2
    ["RV", "EB",],                    # Detection Method
    "2019AJ....157..111Y",           # Reference (use Ads Bibcode)
    "Gaia DR3 4054631788110501888, HD 317844",    # Notes
)

In [14]:
contact_df = add_observation(contact_df,
    "V745 Cas", 	
    [0.0919,005.72221547173	,0.0919],        # RA
    [0.0793,+62.24137780568	,0.0793],        # Dec
    [0.000007, 1.410571, 0.000007],               # Period in days
    [0.0, 0.0, 0.0],                # Eccentricity
    [0.51, 18.31, 0.51],               # M1 in Msun
    [np.nan,np.nan,np.nan],          # M1 sin3i
    [0.28, 10.47, 0.28],               # M2 in Msun
    [np.nan,np.nan,np.nan],             # M2sin3 
    [0.02, 0.57, 0.02],             # q (mass ratio)
    [np.nan,np.nan,np.nan],        # Mass Function
    "B0V, Contact",                 # Type1
    "B1-2 IV/V, Contact",                            # Type2
    ["RV", "EB",],                    # Detection Method
    "2014MNRAS.442.1560C",           # Reference (use Ads Bibcode)
    "Gaia DR3 430491104738148992, HD 1810",    # Notes
)

In [15]:
contact_df = add_observation(contact_df,
    "VFTS 066", 
    [0.0314,084.38788320337	,0.0314],        # RA
    [0.0332,-69.07630004792	,0.0332],        # Dec
    [0.000007, 1.141171, 0.000007],               # Period in days
    [0.0, 0.0, 0.0],                # Eccentricity
    [5.0, 13.0, 7.0],               # M1 in Msun
    [np.nan,np.nan,np.nan],          # M1 sin3i
    [2.8, 6.6, 3.5],               # M2 in Msun
    [np.nan,np.nan,np.nan],             # M2sin3 
    [0.014, 0.523, 0.014],             # q (mass ratio)
    [np.nan,np.nan,np.nan],        # Mass Function
    "B0V, Contact",                 # Type1
    "B1.5III, Contact",                            # Type2
    ["RV", "EB",],                    # Detection Method
    "2020A&A...634A.119M",           # Reference (use Ads Bibcode)
    "Gaia DR3 4657692406776887680, LMC",    # Notes
)

In [16]:
contact_df = add_observation(contact_df,
    "RZ Pyx",
    [0.0151,133.01829794553	,0.0151],        # RA
    [0.0209,-27.48371579951	,0.0209],        # Dec
    [0.000034, 0.656273, 0.000034],          # Period in days
    [0.0, 0.0, 0.0],                # Eccentricity
    [0.4, 5.3, 0.4],                # M1 in Msun
    [np.nan,np.nan,np.nan],          # M1 sin3i
    [0.10, 2.61, 0.10],             # M2 in Msun
    [np.nan,np.nan,np.nan],             # M2sin3 
    [0.02, 0.82, 0.02],             # q (mass ratio)
    [np.nan,np.nan,np.nan],         # Mass Function
    "B5V, Contact",                 # Type1
    "B5/7V, Contact",               # Type2
    ["RV", "EB",],                  # Detection Method
    "1987MNRAS.227..481B",          # Reference (use Ads Bibcode)
    "Gaia DR3 5648950448965966080, HD 75920",    # Notes
)

In [17]:
contact_df = add_observation(contact_df,
    "LY Aur",		
    [0.1388,082.42771093332	,0.1388],        # RA
    [0.0946,+35.37501390189	,0.0946],        # Dec
    [np.nan, 4.02, np.nan],          # Period in days
    [0.0, 0.0, 0.0],                # Eccentricity
    [np.nan, 25.5, np.nan],                # M1 in Msun
    [np.nan,np.nan,np.nan],          # M1 sin3i
    [np.nan, 14.0, np.nan],             # M2 in Msun
    [np.nan,np.nan,np.nan],             # M2sin3 
    [0.007, 0.550, 0.007],             # q (mass ratio)
    [np.nan,np.nan,np.nan],         # Mass Function
    "O9II, Contact",                 # Type1
    "O9III, Contact",               # Type2
    ["RV", "EB",],                  # Detection Method
    "2013A&A...559A..22M",          # Reference (use Ads Bibcode)
    "Gaia DR3 183255985260080896; quadruple",    # Notes
)

In [18]:
contact_df = add_observation(contact_df, 
    "XZ Cep",	
    [0.0155,338.10447177545	,0.0155],        # RA
    [0.0152,+67.15067801114	,0.0152],        # Dec
    [np.nan, 5.097, np.nan],          # Period in days
    [0.0, 0.0, 0.0],                # Eccentricity
    [1.3, 18.7, 1.3],                # M1 in Msun
    [np.nan,np.nan,np.nan],          # M1 sin3i
    [0.5,  9.3, 0.5],             # M2 in Msun
    [np.nan,np.nan,np.nan],             # M2sin3 
    [0.02, 0.50, 0.02],             # q (mass ratio)
    [np.nan,np.nan,np.nan],         # Mass Function
    "O9.5V, Contact",                 # Type1
    "B1III, Contact",               # Type2
    ["RV", "EB",],                  # Detection Method
    "2017A&A...607A..82M",          # Reference (use Ads Bibcode)
    "Gaia DR3 2224860133836098560",    # Notes
)

In [19]:
contact_df = add_observation(contact_df,
    "V348 Car",
    [0.0137,156.74574995552		,0.0137],        # RA
    [0.0140,-57.67575034056		,0.0140],        # Dec
    [np.nan, 5.562107, np.nan],          # Period in days
    [0.0, 0.0, 0.0],                # Eccentricity
    [4, 32, 4],                # M1 in Msun
    [np.nan,np.nan,np.nan],          # M1 sin3i
    [4,  29, 4],             # M2 in Msun
    [np.nan,np.nan,np.nan],             # M2sin3 
    [0.02, 0.90, 0.02],             # q (mass ratio)
    [np.nan,np.nan,np.nan],         # Mass Function
    "B0-B.05III , Contact",                 # Type1
    "B1III, Contact",               # Type2
    ["RV", "EB",],                  # Detection Method
    "1985MNRAS.213...75H",          # Reference (use Ads Bibcode)
    "Gaia DR3 5351710915065605120; HD 90707",    # Notes
)

In [20]:
contact_df = add_observation(contact_df,
    "V729 Cyg",	
    [0.0176,308.09341566800	,0.0176],        # RA
    [0.0198,+41.30524354553	,0.0198],        # Dec
    [0.000028, 6.597887, 0.000028],          # Period in days
    [0.0, 0.0, 0.0],              # Eccentricity
    [2.9, 31.6, 2.9],             # M1 in Msun
    [np.nan,np.nan,np.nan],          # M1 sin3i
    [0.3,  8.8, 0.3],             # M2 in Msun
    [np.nan,np.nan,np.nan],             # M2sin3 
    [0.012, 0.290, 0.012],        # q (mass ratio)
    [np.nan,np.nan,np.nan],       # Mass Function
    "O6.5-7 Ia, Contact",                 # Type1
    "Ofpe/WN9, Contact",               # Type2
    ["RV", "EB",],                  # Detection Method
    "2014NewA...31...32Y",          # Reference (use Ads Bibcode)
    "Gaia DR3 2067830941174418048",    # Notes
)

In [21]:
contact_df = add_observation(contact_df,
    "BH Cen",	
    [0.0104,174.79238064620		,0.0104],        # RA
    [0.0117,-63.42085538671	,0.0117],        # Dec
    [np.nan, 0.79159210, np.nan],          # Period in days
    [0.0, 0.0, 0.0],              # Eccentricity
    [5.4,  9.4, 5.4],             # M1 in Msun
    [np.nan,np.nan,np.nan],          # M1 sin3i
    [5.4,  7.9, 5.4],             # M2 in Msun
    [np.nan,np.nan,np.nan],             # M2sin3 
    [0.003, 0.844, 0.003],        # q (mass ratio)
    [np.nan,np.nan,np.nan],       # Mass Function
    "B, Contact",                 # Type1
    "B, Contact",               # Type2
    ["RV", "EB",],                  # Detection Method
    "1984AJ.....89..872L",          # Reference (use Ads Bibcode)
    "Gaia DR3 5333576497997161728; HD 308826",    # Notes
)

In [22]:
contact_df = add_observation(contact_df,
    "HD 64315B",		
    [0.0561,118.08449889505		,	0.0561],        # RA
    [0.0770,-26.42962363296		,0.0770],        # Dec
    [0.0000008 , 1.0189569, 0.0000008 ],          # Period in days
    [0.0, 0.0, 0.0],              # Eccentricity
    [2.3,  14.6, 2.3],             # M1 in Msun
    [np.nan,np.nan,np.nan],          # M1 sin3i
    [2.3,  14.6, 2.3],             # M2 in Msun
    [np.nan,np.nan,np.nan],             # M2sin3 
    [0.006, 1.00, 0.006],        # q (mass ratio)
    [np.nan,np.nan,np.nan],       # Mass Function
    "O9V, Contact",                 # Type1
    "O9V, Contact",               # Type2
    ["RV", "EB",],                  # Detection Method
    "2017A&A...606A..54L",          # Reference (use Ads Bibcode)
    "Gaia DR3 5602025904044961536; quadruple",    # Notes
)

In [23]:
contact_df = add_observation(contact_df,
    "DIRECT V4741 M31A",		
    [np.nan, 0.083820000	,	np.nan],        # RA
    [0.0770, 41.001049		,np.nan],        # Dec
    [np.nan , 1.604, np.nan ],          # Period in days
    [0.0, 0.0, 0.0],              # Eccentricity
    [np.nan,  18, np.nan],             # M1 in Msun
    [np.nan,np.nan,np.nan],          # M1 sin3i
    [np.nan,  16.8, np.nan],             # M2 in Msun
    [np.nan,np.nan,np.nan],             # M2sin3 
    [0.015, 0.924, 0.015],        # q (mass ratio)
    [np.nan,np.nan,np.nan],       # Mass Function
    "O, Contact",                 # Type1
    "O, Contact",               # Type2
    ["RV", "EB",],                  # Detection Method
    "2022ApJ...932...14L",          # Reference (use Ads Bibcode)
    "[VRJ2006] M31V-J00452011+4145037, M31",    # Notes
)

In [24]:
contact_df = add_observation(contact_df,
    "DIRECT V1555 M31A",		
    [np.nan,0.021779999		,np.nan],        # RA
    [np.nan, 41.646191		,np.nan],        # Dec
    [np.nan , 0.917, np.nan ],          # Period in days
    [0.0, 0.0, 0.0],              # Eccentricity
    [np.nan,  23, np.nan],             # M1 in Msun
    [np.nan,np.nan,np.nan],          # M1 sin3i
    [np.nan,  22.4, np.nan],             # M2 in Msun
    [np.nan,np.nan,np.nan],             # M2sin3 
    [0.012, 0.974, 0.012],        # q (mass ratio)
    [np.nan,np.nan,np.nan],       # Mass Function
    "O, Contact",                 # Type1
    "O, Contact",               # Type2
    ["RV", "EB",],                  # Detection Method
    "2022ApJ...932...14L",          # Reference (use Ads Bibcode)
    "[VRJ2006] M31V-J00450522+4138462, M31",    # Notes
)

In [25]:
contact_df.to_pickle('contact.pkl')